In [4]:
import torch
from diffusers import TextToVideoSDPipeline
from diffusers.utils import export_to_gif

device = "cuda:2"
torch.cuda.set_device(device)

pipe = TextToVideoSDPipeline.from_pretrained(
    "/data/noah/ckpt/pretrain_ckpt/StableDiffusion/modelscope_cerspense", torch_dtype=torch.float16
).to(device)

prompt = "cars driving on a highway"

video_frames = pipe(prompt, height=320, width=576).frames

from PIL import Image

for idx, v in enumerate(video_frames):
    video_frames[idx] = Image.fromarray(v)
export_to_gif(video_frames, output_gif_path="/data/noah/inference/incabin_sample/out/spider.gif")

100%|██████████| 50/50 [00:33<00:00,  1.51it/s]


'/data/noah/inference/incabin_sample/out/spider.gif'

# Video-to-Video


In [3]:
from PIL import Image
import torch
from diffusers import VideoToVideoSDPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
import cv2

device = "cuda:3"
torch.cuda.set_device(device)


def extract_frame(video_path):
    frames = []
    cap = cv2.VideoCapture(video_path)

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # frame = cv2.resize(frame, (256, 256))
        frame = cv2.resize(frame, (576, 320))
        # frame = cv2.resize(frame, (1024, 576))
        frames.append(frame)

    cap.release()
    return frames


pipe = VideoToVideoSDPipeline.from_pretrained(
    "/data/noah/ckpt/pretrain_ckpt/StableDiffusion/modelscope_cerspense", torch_dtype=torch.float16
)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.load_lora_weights("/data/noah/ckpt/finetuning/TEXT2VID_LORA_AD", weight_name="pytorch_lora_weights.safetensors")

# pipe.enable_model_cpu_offload()

# memory optimization
pipe.to(device)

# prompt = "cars are driving on the road"
prompt = ""
input_path = "/data/noah/inference/incabin_sample/reference_vid/sample_2.mp4"
frames = extract_frame(input_path)[:10]

video_frames = pipe(prompt, video=frames, strength=0.6).frames

for idx, v in enumerate(video_frames):
    video_frames[idx] = Image.fromarray(v)
export_to_gif(video_frames, output_gif_path="/data/noah/inference/incabin_sample/out/sample_2.gif")

Loading pipeline components...: 100%|██████████| 5/5 [00:04<00:00,  1.10it/s]
You have saved the LoRA weights using the old format. To convert the old LoRA weights to the new format, you can first load them in a dictionary and then create a new dictionary like the following: `new_state_dict = {f'unet.{module_name}': params for module_name, params in old_state_dict.items()}`.
100%|██████████| 30/30 [00:14<00:00,  2.12it/s]


'/data/noah/inference/incabin_sample/out/sample_2.gif'

: 

# Text-to-Video


In [1]:
import torch
from diffusers import TextToVideoSDPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_gif
import cv2

device = "cuda:3"
torch.cuda.set_device(device)

pipe = TextToVideoSDPipeline.from_pretrained(
    "/data/noah/ckpt/pretrain_ckpt/StableDiffusion/modelscope_cerspense", torch_dtype=torch.float16
)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.load_lora_weights("/data/noah/ckpt/finetuning/TEXT2VID_LORA_AD", weight_name="pytorch_lora_weights.safetensors")
# pipe.enable_model_cpu_offload()

# memory optimization
# pipe.unet.enable_forward_chunking(chunk_size=1, dim=1)
# pipe.enable_vae_slicing()
pipe.to(device)

prompt = "cars are driving on a highway"

video_frames = pipe(prompt, height=320, width=576).frames

from PIL import Image

for idx, v in enumerate(video_frames):
    video_frames[idx] = Image.fromarray(v)
export_to_gif(video_frames, output_gif_path="/data/noah/inference/incabin_sample/out/sample_2.gif")

Loading pipeline components...: 100%|██████████| 5/5 [00:04<00:00,  1.06it/s]
You have saved the LoRA weights using the old format. To convert the old LoRA weights to the new format, you can first load them in a dictionary and then create a new dictionary like the following: `new_state_dict = {f'unet.{module_name}': params for module_name, params in old_state_dict.items()}`.
100%|██████████| 50/50 [00:36<00:00,  1.35it/s]


'/data/noah/inference/incabin_sample/out/sample_2.gif'